In [1]:
import cdsapi
from dotenv import load_dotenv
import os

In [2]:
load_dotenv("copernicus_api.env")

cds = cdsapi.Client(
    url="https://cds.climate.copernicus.eu/api/v2",
    key=f"{os.getenv('UID')}:{os.getenv('API_KEY')}"
)

In [3]:
deg_limit_north = 54
deg_limit_south = 50
deg_limit_west = 2
deg_limit_east = 8

In [4]:
response = cds.retrieve(
    'reanalysis-era5-single-levels',
    {
        "product_type": "reanalysis",
        "format": "grib",
        "variable": "2m_temperature",
        "area": [
            deg_limit_north, deg_limit_west, deg_limit_south, deg_limit_east
        ],
        "year": '2020',
        "month": '03',
        "day": [f"{day:02d}" for day in range(1, 2)],
        "time": [f"{hour:02d}:00" for hour in range(24)]

    }, 'download.grib')

print(response)

2024-03-14 10:31:14,483 INFO Welcome to the CDS
2024-03-14 10:31:14,484 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-03-14 10:31:14,758 INFO Request is queued
2024-03-14 14:06:14,168 INFO Request is completed
2024-03-14 14:06:14,198 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data5/adaptor.mars.internal-1710421492.8809206-23733-3-a028c329-75e9-4edb-9077-11bedc6d27ba.grib to download.grib (22.5K)
2024-03-14 14:06:14,753 INFO Download rate 40.7K/s


Result(content_length=23040,content_type=application/x-grib,location=https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data5/adaptor.mars.internal-1710421492.8809206-23733-3-a028c329-75e9-4edb-9077-11bedc6d27ba.grib)


In [5]:
cds.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'area': [
            54, 9, 53,
            11,
        ],
        'day': [
            '01', '02', '03'
        ],
        'month': '03',
        'time': [
            '00:00','12:00'
        ],
        'year': '2023',
        'variable': '2m_temperature',
    },
    'download.grib')


2024-03-14 14:06:14,868 INFO Welcome to the CDS
2024-03-14 14:06:14,868 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-03-14 14:06:15,107 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data7/adaptor.mars.internal-1710290498.326909-30868-1-06871e77-e87d-4976-b756-dfd98c8c0726.grib to download.grib (1.4K)
2024-03-14 14:06:15,437 INFO Download rate 4.3K/s


Result(content_length=1440,content_type=application/x-grib,location=https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data7/adaptor.mars.internal-1710290498.326909-30868-1-06871e77-e87d-4976-b756-dfd98c8c0726.grib)